# HW1 - A2 : Naive Bayes classifier with missing values handling  

In the bellow code, we have build a naive bayes classifier using normal distribution for each attributes. This code doesn't handle missing (null) values in the dataset.

The function holdout_validation is the function that is used to compute the final accuracy by averaging the accuracy of the classifier over 10 test-train splits.
We yield a final average accuracy of about 75%. 

In [63]:
import pandas as pd
import numpy as np
import math

pima_data = pd.read_csv("pima-indians-diabetes.csv")
pima_data.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.500000,36.600000,0.625000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [64]:
pima_data.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [65]:
# Seperate data in two data frames one for each class (0 and 1)
def seperate_by_class(X, y):
    class_0 = X[y == 0.]
    class_1 = X[y == 1.]
    return class_0, class_1

# Compute mean of elements
def mean(elements):
    return float(sum(elements)) / float(len(elements))

# Standard deviation mean of elements
def standard_deviation(elements):
    m = mean(elements)
    var = sum([math.pow(e - m, 2) for e in elements]) / float(len(elements) - 1)
    return math.sqrt(var)

# Compute Mean and Std_Dev of each attribute for each class
def statistics(X, y):
    classes_statistics = []
    classes = seperate_by_class(X, y)
    for c in classes:
        attribute_stats = [(mean(c[column]), standard_deviation(c[column])) for column in c]
        classes_statistics.append(attribute_stats)
    return classes_statistics

def normal(mean, std_dev, x):
    return math.exp(-0.5 * math.pow((x - mean) / std_dev, 2)) / (math.sqrt(2 * math.pi) * std_dev)  

def train_test_split(data):
    train, test = np.split(data.sample(frac=1), [int(.8*len(data))])
    X_train, y_train = np.split(train, [-1], axis=1)
    y_train = np.squeeze(y_train) # for n x 1 vector
    X_test, y_test = np.split(test, [-1], axis=1)
    y_test = np.squeeze(y_test) # for n x 1 vector
    return X_train, y_train, X_test, y_test

def predict(x, classes_statistics, p_y_0, p_y_1):
    p_c0 = math.log(p_y_0)
    p_c1 = math.log(p_y_1)
    for i in range(len(x)):
        p_c0 += math.log(normal(classes_statistics[0][i][0], classes_statistics[0][i][1], x[i])) # mean and std_dev
        p_c1 += math.log(normal(classes_statistics[1][i][0], classes_statistics[1][i][1], x[i]))
    if p_c0 > p_c1:
        return 0
    else: 
        return 1
    
def compute_accuracy(X_test, y_test, y_train, classes_statistics):
    accuracy = 0
    p_y_0 = float((y_train == 0).sum()) / float(len(y_train))
    p_y_1 = float((y_train == 1).sum()) / float(len(y_train))
    for i, x in X_test.iterrows():
        if predict(x, classes_statistics, p_y_0, p_y_1) == y_test[i]:
            accuracy += 1
    accuracy = accuracy / float(len(y_test))
    return accuracy

def holdout_validation(data):
    final_accuracy = 0
    for i in range(10):
        X, y = np.split(data, [-1], axis=1)
        X_train, y_train, X_test, y_test = train_test_split(data)
        classes_statistics = statistics(X_train, y_train)
        accuracy = compute_accuracy(X_test, y_test, y_train, classes_statistics)
        print "accuracy " + str(accuracy)
        final_accuracy += accuracy
    final_accuracy = final_accuracy / 10.
    print "final accuracy " + str(final_accuracy)
        
holdout_validation(pima_data)

accuracy 0.805194805195
accuracy 0.772727272727
accuracy 0.707792207792
accuracy 0.707792207792
accuracy 0.701298701299
accuracy 0.792207792208
accuracy 0.798701298701
accuracy 0.733766233766
accuracy 0.75974025974
accuracy 0.74025974026
final accuracy 0.751948051948
